In [1]:
import os
import yaml

config = yaml.safe_load(open("myconfig.yml"))

os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
os.environ["WANDB_PROJECT"] = config["WANDB_PROJECT"]

# Tools

In [2]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.tools import tool

In [3]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [4]:
@tool
def calculate_length_tool(a: str) -> int:
    """The function calculates the length of the input string."""
    return len(a)

@tool
def calculate_uppercase_tool(a: str) -> int:
    """The function calculates the number of uppercase characters in the input string."""
    return sum(1 for c in a if c.isupper())

# Memory

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

### Long-term memory

In [6]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "neurons_lab_search",
    "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!",
)

### Short-term memory

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [8]:
message_history = ChatMessageHistory()

### Sensory memory

https://smith.langchain.com/hub/hwchase17/openai-functions-agent

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import hub

In [10]:
prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")

In [11]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace

# Agent

### All tools together

In [12]:
tools = [retriever_tool, tavily_tool, calculate_length_tool, calculate_uppercase_tool]
for t in tools:
    print(t.name, t.description)

neurons_lab_search Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!
tavily_search_results_json A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
calculate_length_tool calculate_length_tool(a: str) -> int - The function calculates the length of the input string.
calculate_uppercase_tool calculate_uppercase_tool(a: str) -> int - The function calculates the number of uppercase characters in the input string.


### Defining an agent with tools and memory

In [13]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [14]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

### Running an agent

In [15]:
agent_with_chat_history.invoke(
    {
        "input": "Hello!"
    },
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'Hello!',
 'chat_history': [],
 'output': 'Hello! How can I assist you today?'}

In [16]:
agent_with_chat_history.invoke(
    {
        "input": "Find the description of Neurons Lab services and calculate the length of this description"
    }, 
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `neurons_lab_search` with `{'query': 'Neurons Lab services description'}`


Neurons Lab | Home


























 



Your vision
Our expertise
Secret sauce*


                            Recipe                            



Partner with us


                                for INNOVATION                                                    
We are AI solution development experts for fast-growing companies seeking innovation through collaboration.






















HealthTech
CleanTech
RetailTech





AWS Certified Advanced partner
and Generative AI competency holder






AI2 = Accelerators2 + 
Handbook2


AI feasibility analysis
Discover


AI solution engineering
Launch


AI operations management
Scale





Success =(Execution x Talent) / 
(Uncertainty x Time)

All  stories

                                    1 — 6                                    











Creative Practice Solutions : Developing an AI-Driven Medical

{'input': 'Find the description of Neurons Lab services and calculate the length of this description',
 'chat_history': [HumanMessage(content='Hello!'),
  AIMessage(content='Hello! How can I assist you today?')],
 'output': 'The description of Neurons Lab services is 620 characters long. If you need more information or assistance, feel free to ask!'}

In [17]:
agent_with_chat_history.invoke(
    {
        "input": "Find the description of Neurons Lab services and calculate the uppercase characters of this description"
    }, 
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `neurons_lab_search` with `{'query': 'description of Neurons Lab services'}`


Neurons Lab | Home


























 



Your vision
Our expertise
Secret sauce*


                            Recipe                            



Partner with us


                                for INNOVATION                                                    
We are AI solution development experts for fast-growing companies seeking innovation through collaboration.






















HealthTech
CleanTech
RetailTech





AWS Certified Advanced partner
and Generative AI competency holder






AI2 = Accelerators2 + 
Handbook2


AI feasibility analysis
Discover


AI solution engineering
Launch


AI operations management
Scale





Success =(Execution x Talent) / 
(Uncertainty x Time)

All  stories

                                    1 — 6                                    











Creative Practice Solutions : Developing an AI-Driven Medi

{'input': 'Find the description of Neurons Lab services and calculate the uppercase characters of this description',
 'chat_history': [HumanMessage(content='Hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='Find the description of Neurons Lab services and calculate the length of this description'),
  AIMessage(content='The description of Neurons Lab services is 620 characters long. If you need more information or assistance, feel free to ask!')],
 'output': 'The description of Neurons Lab services contains 155 uppercase characters. If you have any more questions or need further assistance, feel free to ask!'}

In [18]:
agent_with_chat_history.invoke(
    {
        "input": "What is the weather today in Madrid and calculate the uppercase characters of this description"
    }, 
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather today in Madrid'}`


[{'url': 'https://www.accuweather.com/en/es/madrid/308526/current-weather/308526', 'content': 'Current weather in Madrid, Madrid, Spain. Check current conditions in Madrid, Madrid, Spain with radar, hourly, and more.'}, {'url': 'https://www.timeanddate.com/weather/spain/madrid', 'content': 'Current weather in Madrid and forecast for today, tomorrow, and next 14 days'}, {'url': 'https://www.bbc.com/weather/3117735', 'content': 'Madrid - Weather warnings issued 14-day forecast. Weather warnings issued. Forecast - Madrid. Day by day forecast. Last updated today at 06:57. Today, Sunny and a gentle breeze.'}, {'url': 'https://www.accuweather.com/en/es/madrid/308526/weather-forecast/308526', 'content': 'Find out the current weather, temperature, wind, air quality and allergy outlook for Madrid, Spain. See the hourly and 10-day forecast for sunshine, clouds, showers an

{'input': 'What is the weather today in Madrid and calculate the uppercase characters of this description',
 'chat_history': [HumanMessage(content='Hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='Find the description of Neurons Lab services and calculate the length of this description'),
  AIMessage(content='The description of Neurons Lab services is 620 characters long. If you need more information or assistance, feel free to ask!'),
  HumanMessage(content='Find the description of Neurons Lab services and calculate the uppercase characters of this description'),
  AIMessage(content='The description of Neurons Lab services contains 155 uppercase characters. If you have any more questions or need further assistance, feel free to ask!')],
 'output': 'The weather in Madrid today is described as "Today, Sunny and a gentle breeze." This description contains 2 uppercase characters. If you have any more questions or need further assistance, feel fr

In [19]:
agent_with_chat_history.invoke(
    {
        "input": "Make a quick summary of what we have discussed today"
    }, 
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Today, we discussed finding the description of Neurons Lab services and calculating the length and uppercase characters of the description. We also looked up the weather in Madrid and calculated the uppercase characters in the weather description. If you have any more questions or need further assistance, feel free to let me know!

> Finished chain.


{'input': 'Make a quick summary of what we have discussed today',
 'chat_history': [HumanMessage(content='Hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='Find the description of Neurons Lab services and calculate the length of this description'),
  AIMessage(content='The description of Neurons Lab services is 620 characters long. If you need more information or assistance, feel free to ask!'),
  HumanMessage(content='Find the description of Neurons Lab services and calculate the uppercase characters of this description'),
  AIMessage(content='The description of Neurons Lab services contains 155 uppercase characters. If you have any more questions or need further assistance, feel free to ask!'),
  HumanMessage(content='What is the weather today in Madrid and calculate the uppercase characters of this description'),
  AIMessage(content='The weather in Madrid today is described as "Today, Sunny and a gentle breeze." This description contains 2 

### Structuring outputs

In [20]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [21]:
# Define your desired data structure.
class DesiredStructure(BaseModel):
    question: str = Field(description="the question asked")
    numerical_answer: int = Field(description="the number extracted from the answer, text excluded")
    text_answer: str = Field(description="the text part of the answer, numbers excluded")
parser = PydanticOutputParser(pydantic_object=DesiredStructure)

In [22]:
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_and_model = prompt | model
output = prompt_and_model.invoke({
    "query": "Find the description of Neurons Lab services and calculate the length of this description"}
)

In [23]:
output

AIMessage(content='{\n  "question": "Find the description of Neurons Lab services and calculate the length of this description",\n  "numerical_answer": 63,\n  "text_answer": "Neurons Lab services provide advanced solutions in artificial intelligence, machine learning, and data analytics."\n}', response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 253, 'total_tokens': 309}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

Next stes: evaluation and benchmarks https://python.langchain.com/docs/langsmith/walkthrough